In [9]:
import gspread
import json
from oauth2client.service_account import ServiceAccountCredentials

# (1) Google Spread Sheetsにアクセス
def connect_gspread(jsonf,key):
    scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
    credentials = ServiceAccountCredentials.from_json_keyfile_name(jsonf, scope)
    gc = gspread.authorize(credentials)
    SPREADSHEET_KEY = key
    worksheet = gc.open_by_key(SPREADSHEET_KEY).sheet1
    return worksheet

# ここでjsonfile名と2-2で用意したkeyを入力
jsonf = "test-314711-49ce03b4e35b.json"
spread_sheet_key = "1YEBZnRMQMSUbIYuPFN0KeVJq8Xvs-cgrLlLz7OmRBfw"
ws = connect_gspread(jsonf,spread_sheet_key)

#(2) Google Spread Sheets上の値を更新
#(２−１)あるセルの値を更新（行と列を指定）
ws.update_cell(1,1,"test1")
ws.update_cell(2,1,1)
ws.update_cell(3,1,2)

#(２−２)あるセルの値を更新（ラベルを指定）
ws.update_acell('C1','test2')
ws.update_acell('C2',1)
ws.update_acell('C3',2)

#(2-3)ある範囲のセルの値を更新
ds= ws.range('E1:G3')
ds[0].value = 1
ds[1].value = 2
ds[2].value = 3
ds[3].value = 4
ds[4].value = 5
ds[5].value = 6
ds[6].value = 7
ds[7].value = 8
ds[8].value = 9
ws.update_cells(ds)


{'spreadsheetId': '1YEBZnRMQMSUbIYuPFN0KeVJq8Xvs-cgrLlLz7OmRBfw',
 'updatedRange': "'シート1'!E1:G3",
 'updatedRows': 3,
 'updatedColumns': 3,
 'updatedCells': 9}

In [12]:
import pandas as pd

In [30]:
df= pd.DataFrame(
    data=[[1,2],["インターンの頑張り方が分からん^_^","https://Twitter.com/ppi____17/status/1395022997254070273"],["そろそろインターンとかシュ…活しないとね、まずいよね、わかっとる","https://Twitter.com/i___iiiiiiii/status/1395017434239705088"]],
    columns=["テキスト","URL"]
    )

In [31]:
df

,テキスト,URL
0,1,2
1,インターンの頑張り方が分からん^_^,https://Twitter.com/ppi____17/status/139502299...
2,そろそろインターンとかシュ…活しないとね、まずいよね、わかっとる,https://Twitter.com/i___iiiiiiii/status/139501...


In [26]:
ds= ws.range('A1:B2')
ds[0].value = df["テキスト"][0]
ds[1].value = df["URL"][0]
ds[2].value = df["テキスト"][1]
ds[3].value = df["URL"][1]
ds[4].value = df["テキスト"][2]
ds[5].value = df["URL"][2]
ws.update_cells(ds)

{'spreadsheetId': '1YEBZnRMQMSUbIYuPFN0KeVJq8Xvs-cgrLlLz7OmRBfw',
 'updatedRange': "'シート1'!A1:B2",
 'updatedRows': 2,
 'updatedColumns': 2,
 'updatedCells': 4}

In [32]:
ds= ws.range("A1:B{}".format(len(df)))
for i in range(len(df)):
    ds[2*i].value = df["テキスト"][i]
    ds[2*i+1].value = df["URL"][i]
ws.update_cells(ds)

{'spreadsheetId': '1YEBZnRMQMSUbIYuPFN0KeVJq8Xvs-cgrLlLz7OmRBfw',
 'updatedRange': "'シート1'!A1:B3",
 'updatedRows': 3,
 'updatedColumns': 2,
 'updatedCells': 6}

In [4]:
import gspread
import pandas as pd
from oauth2client.service_account import ServiceAccountCredentials
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive

scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']
json_keyfile_path = 'test-314711-49ce03b4e35b.json'

# サービスアカウントキーを読み込む
credentials = ServiceAccountCredentials.from_json_keyfile_name(
    json_keyfile_path, scope)

# pydrive用にOAuth認証を行う
gauth = GoogleAuth()
gauth.credentials = credentials
drive = GoogleDrive(gauth)

folder_id = '17jAyV_RrfXaHZ4_-ptY0Benf-aeGp8Qi'
f = drive.CreateFile({
    'title': 'sample_spread',
    'mimeType': 'application/vnd.google-apps.spreadsheet',
    "parents": [{"id": folder_id}]})
f.Upload()

# gspread用に認証
gc = gspread.authorize(credentials)

# スプレッドシートのIDを指定してワークブックを選択
workbook = gc.open_by_key(f['id'])
worksheet = workbook.sheet1

df= pd.DataFrame(
    data=[[1,2],["インターンの頑張り方が分からん^_^","https://Twitter.com/ppi____17/status/1395022997254070273"],["そろそろインターンとかシュ…活しないとね、まずいよね、わかっとる","https://Twitter.com/i___iiiiiiii/status/1395017434239705088"]],
    columns=["テキスト","URL"]
    )

cell_list = worksheet.range("A1:B{}".format(len(df)))
for i in range(len(df)):
    cell_list[2*i].value = df["テキスト"][i]
    cell_list[2*i+1].value = df["URL"][i]

# スプレッドシートを更新
worksheet.update_cells(cell_list)

{'spreadsheetId': '17yX6sF98SIEGbTheO5X26fnrKNHw83xHnVjEkb8BIKI',
 'updatedRange': 'Sheet1!A1:B3',
 'updatedRows': 3,
 'updatedColumns': 2,
 'updatedCells': 6}